# (4주차) 9월27일 -- draft
> 딥러닝의 기초 - 회귀분석(2)--step1~4, step 2,3,4의 다른표현

- toc:true
- branch: master
- badges: true
- comments: true
- author: 최규빈

## Imports

In [387]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

## numpy, torch (선택학습)

### numpy, torch는 엄청 비슷해요 

`-` torch.tensor() = np.array() 처럼 생각해도 무방 

In [444]:
np.array([1,2,3]), torch.tensor([1,2,3])

(array([1, 2, 3]), tensor([1, 2, 3]))

`-` 소수점의 정밀도에서 차이가 있음 (torch가 좀 더 쪼잔함) 

In [445]:
np.array([3.123456789])

array([3.12345679])

In [446]:
torch.tensor([3.123456789])

tensor([3.1235])

`-` 기본적인 numpy 문법은 np 대신에 torch를 써도 무방 // 완전 같지는 않음 

In [425]:
np.arange(10), torch.arange(10)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]))

In [426]:
np.linspace(0,1,10), torch.linspace(0,1,10)

(array([0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
        0.55555556, 0.66666667, 0.77777778, 0.88888889, 1.        ]),
 tensor([0.0000, 0.1111, 0.2222, 0.3333, 0.4444, 0.5556, 0.6667, 0.7778, 0.8889,
         1.0000]))

In [428]:
np.random.randn(10)

array([-1.65005215,  0.6765417 ,  0.2340635 , -0.23399836,  0.24460602,
        0.26614919,  0.12982052, -0.31751179,  0.77568713,  0.00523241])

In [431]:
torch.randn(10)

tensor([-0.6582, -0.6651, -0.9028,  1.2691, -1.5274,  0.0752, -2.2186,  0.0333,
        -0.1234,  0.2954])

### length $n$ vector, $n \times 1$ col-vector, $1 \times n$ row-vector 

`-` 길이가 3인 벡터 선언방법 

In [513]:
a = torch.tensor([1,2,3])
a.shape

torch.Size([3])

`-` 3x1 col-vec 선언방법 

(방법1)

In [514]:
a = torch.tensor([[1],[2],[3]])
a.shape

torch.Size([3, 1])

(방법2)

In [515]:
a = torch.tensor([1,2,3]).reshape(3,1)
a.shape

torch.Size([3, 1])

`-` 1x3 row-vec 선언방법 

(방법1)

In [517]:
a = torch.tensor([[1,2,3]])
a.shape

torch.Size([1, 3])

(방법2)

In [518]:
a = torch.tensor([1,2,3]).reshape(1,3)
a.shape

torch.Size([1, 3])

`-` 3x1 col-vec 선언방법, 1x3 row-vec 선언방법에서 `[[1],[2],[3]]` 혹은 `[[1,2,3]]` 와 같은 표현이 이해안되면 아래링크로 가셔서 

https://guebin.github.io/STBDA2022/2022/03/14/(2주차)-3월14일.html

첫번째 동영상 12:15 - 22:45 에 해당하는 분량을 학습하시길 바랍니다. 

### torch의 dtype

`-` 기본적으로 torch는 소수점으로 저장되면 dtype=torch.float32 가 된다. (이걸로 맞추는게 편리함) 

In [491]:
tsr = torch.tensor([1.23,2.34])
tsr

tensor([1.2300, 2.3400])

In [492]:
tsr.dtype

torch.float32

`-` 정수로 선언하더라도 dtype를 torch.float32로 바꾸는게 유리함 

(안 좋은 선언예시) 

In [493]:
tsr = torch.tensor([1,2])
tsr 

tensor([1, 2])

In [494]:
tsr.dtype

torch.int64

(좋은 선언예시1) 

In [504]:
tsr = torch.tensor([1,2],dtype=torch.float32)
tsr 

tensor([1., 2.])

In [505]:
tsr.dtype

torch.float32

(좋은 선언예시2) 

In [507]:
tsr = torch.tensor([1,2.0])
tsr 

tensor([1., 2.])

In [508]:
tsr.dtype

torch.float32

(사실 int로 선언해도 나중에 float으로 바꾸면 큰 문제없음) 

In [522]:
tsr = torch.tensor([1,2]).float()
tsr

tensor([1., 2.])

In [523]:
tsr.dtype

torch.float32

`-` 왜 정수만으로 torch.tensor를 만들때에도 torch.float32로 바꾸는게 유리할까? $\to$ torch.tensor끼리의 연산에서 문제가 될 수 있음 

별 문제 없을수도 있지만 

In [502]:
torch.tensor([1,2])-torch.tensor([1.0,2.0]) 

tensor([0., 0.])

에러가 날수도 있다 

In [503]:
torch.tensor([[1.0,0.0],[0.0,1.0]]) @ torch.tensor([[1],[2]]) 

RuntimeError: expected scalar type Float but found Long

In [499]:
torch.tensor([[1,0],[0,1]]) @ torch.tensor([[1.0],[2.0]])

RuntimeError: expected scalar type Long but found Float

둘다 정수로 통일하는것도 방법이지만 

In [500]:
torch.tensor([[1,0],[0,1]]) @ torch.tensor([[1],[2]])

tensor([[1],
        [2]])

둘다 소수로 통일하는게 더 좋음 

In [501]:
torch.tensor([[1.0,0.0],[0.0,1.0]]) @ torch.tensor([[1.0],[2.0]])

tensor([[1.],
        [2.]])

### shape of vector

`-` 행렬곱셈에 대한 shape 조심

In [463]:
A = torch.tensor([[2.00,0.00],[0.00,3.00]]) 
b1 = torch.tensor([[-1.0,-5.0]])
b2 = torch.tensor([[-1.0],[-5.0]])
b3 = torch.tensor([-1.0,-5.0])

In [464]:
A.shape,b1.shape,b2.shape,b3.shape

(torch.Size([2, 2]), torch.Size([1, 2]), torch.Size([2, 1]), torch.Size([2]))

`-` A@b1: 계산불가, b1@A: 계산가능

In [467]:
A@b1

RuntimeError: mat1 and mat2 shapes cannot be multiplied (2x2 and 1x2)

In [468]:
b1@A

tensor([[ -2., -15.]])

`-` A@b2: 계산가능, b2@A: 계산불가 

In [470]:
A@b2

tensor([[ -2.],
        [-15.]])

In [471]:
b2@A

RuntimeError: mat1 and mat2 shapes cannot be multiplied (2x1 and 2x2)

`-` A@b3: 계산가능, b3@A: 계산가능 

In [475]:
(A@b3).shape ## b3를 마치 col-vec 처럼 해석

torch.Size([2])

In [476]:
(b3@A).shape ## b3를 마지 row-vec 처럼 해석

torch.Size([2])

`-` 브로드캐스팅 

In [459]:
a = torch.tensor([1,2,3])
a - 1

tensor([0, 1, 2])

In [460]:
b = torch.tensor([[1],[2],[3]])
b - 1

tensor([[0],
        [1],
        [2]])

In [462]:
a - b # a를 row-vec 로 해석

tensor([[ 0,  1,  2],
        [-1,  0,  1],
        [-2, -1,  0]])

## 지난시간복습

## step2의 다른버전: torch.nn.MSELoss()

yhat은 어떠한 방식으로든 만들었다고 가정하자. 

In [395]:
What = torch.tensor([[-5.0],[10.0]],requires_grad=True)
yhat = X@What

### ver1

In [396]:
loss_fn = torch.nn.MSELoss()

In [397]:
loss = loss_fn(y,yhat)
loss

tensor(85.8769, grad_fn=<MseLossBackward0>)

### 잘못된사용1 

In [398]:
_y = y.reshape(-1)
_y

tensor([-8.5420, -6.5767, -5.9496, -4.4794, -4.2516, -3.1326, -4.0239, -4.1862,
        -3.3403, -2.2027, -2.0262, -2.5619, -1.3353, -2.0466, -0.4664, -1.3513,
        -1.6472, -0.1089, -0.3071, -0.6299, -0.0438,  0.4163,  0.4166, -0.0943,
         0.2662,  0.4591,  0.8905,  0.8998,  0.6314,  1.3845,  0.8085,  1.2594,
         1.1211,  1.9232,  1.0619,  1.3552,  2.1161,  1.1437,  1.6245,  1.7639,
         1.6022,  1.7465,  0.9830,  1.7824,  2.1116,  2.8621,  2.1165,  1.5226,
         2.5572,  2.8361,  3.3956,  2.0679,  2.8140,  3.4852,  3.6059,  2.5966,
         2.8854,  3.9173,  3.6527,  4.1029,  4.3125,  3.4026,  3.2180,  4.5686,
         4.3772,  4.3075,  4.4895,  4.4827,  5.3170,  5.4987,  5.4632,  6.0328,
         5.2842,  5.0539,  5.4538,  6.0337,  5.7250,  5.7587,  6.2020,  6.5992,
         6.4621,  6.5140,  6.6846,  7.3497,  8.0909,  7.0794,  6.8667,  7.4229,
         7.2544,  7.1967,  9.5006,  9.0339,  7.4887,  9.0759, 11.0946, 10.3260,
        12.2665, 13.0983, 12.5468, 13.83

In [399]:
loss_fn(_y,yhat)

tensor(176.2661, grad_fn=<MseLossBackward0>)

하여튼 shape을 조심해야함!!

## step3의 다른버전: 없어요 ㅎㅎ 

`-` 미분은 무조건 `loss.backward()`로 하면된다. 

`-` 그런데 step1에서 정의한 다른 방식들에 의해서 미분된값이 저장되는 루트는 다름 

### 예시1: 지난시간에 한거

In [339]:
What = torch.tensor([[-5.0],[10.0]],requires_grad=True)
yhat = X@What
loss = loss_fn(y,yhat) 
loss.backward() 

In [340]:
What.grad

tensor([[-13.4225],
        [ 11.8893]])

### 예시2: (ver1)

`-` net를 생성 

In [364]:
torch.manual_seed(43052)
net = torch.nn.Linear(in_features=1, out_features=1, bias=True) 

`-` net의 weight와 bias가 $y_i=\hat{w}x_i + \hat{b}=10x_i - 5$와 같이 되도록 설정 

In [365]:
net.weight # what = -0.3467 

Parameter containing:
tensor([[-0.3467]], requires_grad=True)

In [366]:
net.weight.data = torch.tensor([[10.00]]) 

In [367]:
net.weight # 이제 what = 10.00 

Parameter containing:
tensor([[10.]], requires_grad=True)

In [368]:
net.bias # 지금 bhat = -0.8470

Parameter containing:
tensor([-0.8470], requires_grad=True)

In [369]:
net.bias.data = torch.tensor([-5.0]) # 이제 bhat = -5.0

In [370]:
net.bias

Parameter containing:
tensor([-5.], requires_grad=True)

`-` yhat계산 $\to$ loss계산 $\to$ 미분 $\to$ 미분값확인 

In [371]:
yhat = net(x) 

In [372]:
loss = loss_fn(y,yhat) 
loss

tensor([[85.8769]], grad_fn=<DivBackward0>)

In [373]:
loss.backward()

In [376]:
net.bias.grad,net.weight.grad

(tensor([-13.4225]), tensor([[11.8893]]))

### 예시3: (ver2) 

`-` net를 생성 

In [377]:
torch.manual_seed(43052)
net = torch.nn.Linear(in_features=2, out_features=1, bias=False) 

`-` net의 weight와 bias가 $y_i=\hat{w}x_i + \hat{b}=10x_i - 5$와 같이 되도록 설정 

In [378]:
net.weight 

Parameter containing:
tensor([[-0.2451, -0.5989]], requires_grad=True)

In [379]:
net.weight.data = torch.tensor([[-5.00,10.00]]) 

In [381]:
net.weight 

Parameter containing:
tensor([[-5., 10.]], requires_grad=True)

`-` yhat계산 $\to$ loss계산 $\to$ 미분 $\to$ 미분값확인 

In [382]:
yhat = net(X) 

In [383]:
loss = loss_fn(y,yhat) 
loss

tensor([[85.8769]], grad_fn=<DivBackward0>)

In [384]:
loss.backward()

In [385]:
net.weight.grad

tensor([[-13.4225,  11.8893]])